## 0 Imports and setup

In [1]:
import logging
from pathlib import Path

import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from scipy.sparse import issparse, csr_matrix
from scipy.stats import median_abs_deviation
from scipy.io import mmwrite

import scanpy as sc

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro
import anndata2ri

import subprocess as sp

sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=300,
    facecolor="white",
     # color_map="YlGnBu",
    frameon=False,
)

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
%%R
#QC
library(Seurat)
library(scater)
library(scDblFinder)
library(BiocParallel)
#normalisation
library(scran)
#feature selection
library(scry)

In [ ]:
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

def write_output(adata, output_dir : Path, identifier : str):
    adata.write(output_dir/f"{identifier}.h5ad") #save h5ad file
    mtx_dir = output_dir/f"{identifier}_mtx"
    mtx_dir.mkdir(exist_ok=True)
    features=adata.var
    barcodes=adata.obs
    features.index.rename("GeneName",inplace=True)
    barcodes.index.rename("Barcode",inplace=True)
    features.to_csv(mtx_dir/"features.tsv",sep="\t",header=False)
    barcodes.to_csv(mtx_dir/"barcodes.tsv",sep="\t",header=False)
    features.to_csv(mtx_dir/"features_header.tsv",sep="\t",header=True)
    barcodes.to_csv(mtx_dir/"barcodes_header.tsv",sep="\t",header=True)
    mmwrite(mtx_dir/"matrix.mtx",csr_matrix(adata.X.T),field="integer")
    
    sp.run(f"gzip -f {str(mtx_dir)}/matrix.mtx",shell=True)
    sp.run(f"gzip -f {str(mtx_dir)}/features.tsv",shell=True)
    sp.run(f"gzip -f {str(mtx_dir)}/barcodes.tsv",shell=True)
    

In [ ]:
cr_dir = Path(globals()["__vsc_ipynb_file__"]).parent.absolute() #globals()["__vsc_ipynb_file__"]
cr_od = str(cr_dir)
s_id = cr_dir.name
outdir= cr_dir.parent / f"cellranger_post/{s_id}/"
if not outdir.is_dir():
    outdir.mkdir()

## 1 QC

### 1.1 SoupX
* Replaced by Cellbender (before notebook execution)

### 1.2 Load cellranger output and add SoupX layer

In [ ]:
adata = sc.read_10x_h5(cr_dir/"feature_bc_matrix_filtered.h5")
adata.var_names_make_unique()
adata.layers["counts"] = adata.X
adata

### 1.3 Filter low-quality cells

#### 1.3.1 Annotate genes and calculate QC metrics

In [12]:
adata.var["mt"] = adata.var_names.str.startswith("MT-") # mitochondrial genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL")) # ribosomal genes
adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]") # hemoglobin genes.
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True)

#### 1.3.2 Plot QC metrics

In [ ]:
fig,axs = plt.subplots(2,2,figsize=(16,16))
sns.histplot(adata.obs["total_counts"], bins=100,ax=axs[0,0])
sns.violinplot(adata.obs,y="total_counts",ax=axs[0,1],inner="box")
sns.stripplot(adata.obs,y="total_counts",ax=axs[0,1],color="black",alpha=0.5,s=2)
axs[0,1].set_ylim(0,axs[0,1].get_ylim()[1])
sns.violinplot(adata.obs,y="pct_counts_mt",ax=axs[1,0],inner="box")
sns.stripplot(adata.obs,y="pct_counts_mt",ax=axs[1,0],color="black",alpha=0.5,s=2)
axs[1,0].set_ylim(0,axs[1,0].get_ylim()[1])

axs[1,1] = plt.scatter(x=adata.obs["total_counts"],y=adata.obs["n_genes_by_counts"],c=adata.obs["pct_counts_mt"],cmap=sns.color_palette("viridis", as_cmap=True),s=10)
plt.colorbar()

sns.despine()
plt.tight_layout()
plt.savefig(outdir/f"{s_id}_pre_filtering.png",dpi=300)
plt.show()

#### 1.3.3 Detect and remove outlier cells

In [ ]:
adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", 5)
    | is_outlier(adata, "log1p_n_genes_by_counts", 5)
    | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
)
print("Cell count outliers:")
print(adata.obs.outlier.value_counts())
print()
adata.obs["mt_outlier"] = is_outlier(adata, "pct_counts_mt", 3) | (
    adata.obs["pct_counts_mt"] > 8
)
print("Mitochondrial counts outliers:")
print(adata.obs.mt_outlier.value_counts())

In [ ]:
print(f"Total number of cells: {adata.n_obs}")
adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()

print(f"Number of cells after filtering of low quality cells: {adata.n_obs}")
p1 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")
plt.show()

### 1.4 Removal of likely background genes

In [ ]:
print(f"Total number of genes: {adata.n_vars}")
# Min 20 cells - filters out 0 count genes
sc.pp.filter_genes(adata, min_cells=20)
print(f"Number of genes after cell filter: {adata.n_vars}")

### 1.5 Doublet detection

In [ ]:
data_mat = adata.X.T.astype(np.float32)

In [18]:
%%R -i data_mat -o doublet_score -o doublet_class

set.seed(123)
sce = scDblFinder(
    SingleCellExperiment(
        list(counts=data_mat),
    ) 
)
doublet_score = sce$scDblFinder.score
doublet_class = sce$scDblFinder.class

In [ ]:
adata.obs["scDblFinder_score"] = doublet_score
adata.obs["scDblFinder_class"] = doublet_class
adata.obs.scDblFinder_class.value_counts()

In [ ]:
adata = adata[adata.obs.scDblFinder_class == "singlet"].copy()

### 1.6 Save QC-passed h5ad file to disk (Optional)

In [34]:
write_output(adata,outdir,"filtered")

gzip: /mnt/shared/workspace/snakemake_rnaseq/results/notebook_test/filtered_mtx/matrix.mtx.gz already exists;	not overwritten
gzip: /mnt/shared/workspace/snakemake_rnaseq/results/notebook_test/filtered_mtx/features.tsv.gz already exists;	not overwritten


## 2 Count Normalisation

### 2.1 Shifted Logarithm

In [16]:
scales_counts = sc.pp.normalize_total(adata, target_sum=None, inplace=False)
adata.layers["log1p_norm"] = sc.pp.log1p(scales_counts["X"], copy=True)

#### 2.1.1 Scran

In [ ]:
adata_pp = adata.copy()
sc.pp.normalize_total(adata_pp)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp, n_comps=15)
sc.pp.neighbors(adata_pp)
sc.tl.leiden(adata_pp, key_added="groups",flavor="igraph",n_iterations=2)
data_mat = adata_pp.X.T
# convert to CSC if possible. See https://github.com/MarioniLab/scran/issues/70
if issparse(data_mat):
    if data_mat.nnz > 2**31 - 1:
        data_mat = data_mat.tocoo()
    else:
        data_mat = data_mat.tocsc()
ro.globalenv["data_mat"] = data_mat
ro.globalenv["input_groups"] = adata_pp.obs["groups"]
del adata_pp

In [18]:
%%R -o size_factors
size_factors = sizeFactors(
    computeSumFactors(
        SingleCellExperiment(
            list(counts=data_mat)), 
            clusters = input_groups,
            min.mean = 0.1,
            BPPARAM = MulticoreParam()
    )
)

In [19]:
adata.obs["size_factors"] = size_factors
scran = adata.X / adata.obs["size_factors"].values[:, None]
adata.layers["scran_normalization"] = csr_matrix(sc.pp.log1p(scran.tocsc()))

### 2.2 Analytic Pearson residuals

In [ ]:
analytic_pearson = sc.experimental.pp.normalize_pearson_residuals(adata, inplace=False)
adata.layers["analytic_pearson_residuals"] = csr_matrix(analytic_pearson["X"])

### 2.3 Visualisation

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
sns.histplot(adata.obs["total_counts"], bins=1000, kde=False, ax=axes[0,0],legend=False)
axes[0,0].set_xlabel("")
sns.histplot(adata.layers["log1p_norm"].sum(1), bins=1000, kde=False, ax=axes[0,1],legend=False)
sns.histplot(adata.layers["scran_normalization"].sum(1), bins=1000, kde=False, ax=axes[1,0],legend=False)
sns.histplot(adata.layers["analytic_pearson_residuals"].sum(1), bins=1000, kde=False, ax=axes[1,1],legend=False)
axes[0,0].set_title("Total counts")
axes[0,1].set_title("Shifted logarithm")
axes[1,0].set_title("log1p with Scran estimated size factors")
axes[1,1].set_title("Analytic Pearson residuals")
sns.despine()
plt.tight_layout()
plt.show()

### 2.4 Save Normalised counts (Optional)

In [22]:
write_output(adata,outdir,"filtered_norm")

## 3 Feature selection

In [ ]:
adata = sc.read(
    filename=outdir / "filtered_norm.h5ad"
)
adata.X = adata.X.astype(np.float32)
adata.layers["counts"] = adata.layers["counts"].astype(np.float32)
ro.globalenv["adata"] = adata

In [26]:
%%R
sce = devianceFeatureSelection(adata, assay="X")

In [28]:
binomial_deviance = ro.r("rowData(sce)$binomial_deviance").T
idx = binomial_deviance.argsort()[-min(int(adata.n_vars*0.2),4000):] #modified to scale better when few genes are present
mask = np.zeros(adata.var_names.shape, dtype=bool)
mask[idx] = True

adata.var["highly_deviant"] = mask
adata.var["binomial_deviance"] = binomial_deviance
#sc.pp.highly_variable_genes(adata, layer="log1p_norm")

In [ ]:
sc.pp.highly_variable_genes(adata, layer="log1p_norm")
ax = sns.scatterplot(
    data=adata.var, x="means", y="dispersions", hue="highly_deviant", s=5
)
plt.show()

## 4 Dimensionality reduction

In [30]:
#adata.X = adata.layers["log1p_norm"]
# setting highly variable as highly deviant to use scanpy 'use_highly_variable' argument in sc.pp.pca

### 4.1 PCA

In [ ]:
sc.pp.pca(adata, svd_solver="arpack", mask_var="highly_deviant")
sc.pl.pca_scatter(adata, color="total_counts")

### 4.2 tSNE

In [ ]:
sc.tl.tsne(adata, use_rep="X_pca")
sc.pl.tsne(adata, color="total_counts")

### 4.3 UMAP

In [ ]:
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.pl.umap(adata, color="total_counts")

### QC metrics

In [ ]:
sc.pl.tsne(
    adata,
    color=["total_counts", "pct_counts_mt", "scDblFinder_score", "scDblFinder_class"],
)